# Setup cell

In [ ]:
# Setup cell
import os

from dotenv import load_dotenv
import pandas as pd
import logging

import bitbucket
from git_operations import compare_git_clone_speed
from metrics import Metrics
from platform_analysis import get_most_present_owner, get_unique_owner_number
from platforms import Platforms
import github
import gitea_forgejo
import gitlab
import platform_analysis

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables from .env file
load_dotenv()

# Load tokens from environment variables
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITLAB_TOKEN = os.getenv("GITLAB_TOKEN")
GITEA_TOKEN = os.getenv("GITEA_TOKEN")
FORGEJO_TOKEN = os.getenv("FORGEJO_TOKEN")
BITBUCKET_TOKEN = os.getenv("BITBUCKET_TOKEN")

# Set up headers for API requests
GITHUB_HEADERS = {"Authorization": f"token {GITHUB_TOKEN}"}
GITEA_HEADERS = {"Authorization": f"token {GITEA_TOKEN}"}
FORGEJO_HEADERS = {"Authorization": f"token {FORGEJO_TOKEN}"}
GITLAB_HEADERS = {"Authorization": f"Bearer {GITLAB_TOKEN}"}
BITBUCKET_HEADERS = {"Authorization": f"Bearer {BITBUCKET_TOKEN}"}

# Initialize platform instances
github_instance = github.GitHub(GITHUB_HEADERS)
gitea_instance = gitea_forgejo.GiteaForgejo(GITEA_HEADERS)
forgejo_instance = gitea_forgejo.GiteaForgejo(FORGEJO_HEADERS)
gitlab_instance = gitlab.Gitlab(GITLAB_HEADERS)
bitbucket_instance = bitbucket.Bitbucket(BITBUCKET_HEADERS)

# Code Demo

In [ ]:
# Fetch a given number of GitHub repositories
data = github_instance.fetch_repositories(50)
file_name = "temp.csv"

# Save the gathered data to a CSV file
if data:
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False)
    logger.info(f"Saved repositories to '{file_name}'.")
else:
    logger.error("No repositories were fetched.")

In [ ]:
input_file = "temp.csv"
output_file = "temp2.csv"
# Load the csv file that contains the repositories
df = pd.read_csv(input_file)
final_df = pd.read_csv(output_file)

df = github_instance.select_clonable_repositories(df, Platforms.GITHUB, 10)

# Save updated CSV
df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

In [ ]:
input_file = "temp2.csv"
output_file = "temp2.csv"

# Load the csv file that contains the repositories
df = pd.read_csv(input_file)

# Add a chosen metric for each repository of the DataFrame
github_instance.add_metric(df, Metrics.ISSUE)
github_instance.add_metric(df, Metrics.PULL_REQUEST)

# Save updated CSV
df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

# Analysis Code

In [ ]:
import time

start_time = time.time()  # Record start time

github_instance.fetch_repositories(5000)

end_time = time.time()  # Record end time
duration = end_time - start_time  # Calculate duration

print(f"Function executed in {duration:.2f} seconds.")

In [ ]:
input_file = "temp.csv"
output_file = "temp2.csv"

df = pd.read_csv(input_file)
compare_git_clone_speed(df, Platforms.GITHUB)

# Save updated CSV
df.to_csv(output_file, index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

input_file = "temp2.csv"

df = pd.read_csv(input_file)

# Example: df is your dataframe with 'shallow_clone_time' and 'full_clone_time'
faster_method = df['shallow_clone_time'] < df['full_clone_time']
# Count how many were faster for shallow or full
shallow_faster = faster_method.sum()
full_faster = len(df) - shallow_faster

# Convert to percentages
total = len(df)
percentages = {
    'Shallow clone': (shallow_faster / total) * 100,
    'Clone complet': (full_faster / total) * 100
}


# Plot
plt.bar(percentages.keys(), percentages.values(), color=['skyblue', 'salmon'])
plt.ylabel('Pourcentage de dépôts (%)')
plt.ylim(0, 100)

# Show percentages on top of bars
#for i, v in enumerate(percentages.values()):
    #plt.text(i, v + 1, f"{v:.1f}%", ha='center', fontweight='bold')

plt.savefig('Figures/clone_speed.png', dpi=300, bbox_inches='tight')

plt.show()

print("Somme des temps de clonage :")
print(df[['shallow_clone_time', 'full_clone_time']].sum())



# Fetching Repositories from Github

In [ ]:
# Fetch a given number of GitHub repositories
data = github_instance.fetch_repositories(5000)
file_name = "Github_data/github_5k_repositories.csv"

# Save the gathered data to a CSV file
if data:
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False)
    logger.info(f"Saved repositories to '{file_name}'.")
else:
    logger.error("No repositories were fetched.")

### Select Clonable Github repositories and add Git metrics to the dataframe

In [ ]:
input_file = "Github_data/github_5k_repositories.csv"
output_file = "Github_data/github_repositories_metrics.csv"
# Load the csv file that contains the repositories
df = pd.read_csv(input_file)
final_df = pd.read_csv(output_file)

df = github_instance.select_clonable_repositories(df, Platforms.GITHUB, 2, final_df)

# Save updated CSV
df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

### Add Platform specific metrics to the dataframe

In [ ]:
input_file = "temp2.csv"
output_file = "temp2.csv"

# Load the csv file that contains the repositories
df = pd.read_csv(input_file)

# Add a chosen metric for each repository of the DataFrame
github_instance.add_metric(df, Metrics.ISSUE)
github_instance.add_metric(df, Metrics.PULL_REQUEST)

# Save updated CSV
df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

# Fetching Repositories from Gitea

In [ ]:
# Fetch a given number of pages of 50 Gitea repositories
data = gitea_instance.fetch_repositories(1000, platform=Platforms.GITEA)
file_name = "Gitea_data/gitea_1k_repositories.csv"

# Save the gathered data to a CSV file
if data:
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False)
    logger.info(f"Saved repositories to '{file_name}'.")
else:
    logger.error("No repositories were fetched.")

### Select Clonable Gitea repositories and add Git metrics to the dataframe

In [ ]:
input_file = "Gitea_data/gitea_2k_repositories.csv"
output_file = "Gitea_data/gitea_repositories_metrics.csv"

df = pd.read_csv(input_file)
final_df = pd.read_csv(output_file)
df = gitea_instance.select_clonable_repositories(df, Platforms.GITEA, 100, final_df)
df.to_csv(output_file, index=False)

logger.info(f"Done! Updated file saved as {output_file}")

### Add Platform specific metrics to the dataframe

In [ ]:
input_file = "Gitea_data/gitea_repositories_metrics.csv"
output_file = "Gitea_data/gitea_repositories_metrics.csv"

# Load the csv file that contains the repositories
df = pd.read_csv(input_file)

# Add chosen metric for each repository of the DataFrame
gitea_instance.add_metric(df, Platforms.GITEA, Metrics.ISSUE)
gitea_instance.add_metric(df, Platforms.GITEA, Metrics.PULL_REQUEST)

# Save updated CSV
df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

# Fetching Repositories from Forgejo

In [ ]:
# Fetch a given number of pages of 50 Forgejo repositories
data = forgejo_instance.fetch_repositories(5000, platform=Platforms.FORGEJO)
file_name = "Forgejo_data/forgejo_5k_repositories.csv"

# Save the gathered data to a CSV file
if data:
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False)
    logger.info(f"Saved repositories to '{file_name}'.")
else:
    logger.error("No repositories were fetched.")

### Select Clonable Forgejo repositories and add Git metrics to the dataframe

In [ ]:
input_file = "Forgejo_data/forgejo_5k_repositories.csv"
output_file = "Forgejo_data/forgejo_repositories_metrics.csv"

df = pd.read_csv(input_file)
final_df = pd.read_csv(output_file)

df = forgejo_instance.select_clonable_repositories(df, Platforms.FORGEJO, 50, final_df)

df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

### Add Platform specific metrics to the dataframe

In [ ]:
input_file = "Forgejo_data/forgejo_repositories_metrics.csv"
output_file = "Forgejo_data/forgejo_repositories_metrics.csv"

# Load the csv file that contains the repositories
df = pd.read_csv(input_file)

# Add chosen metric for each repository of the DataFrame
forgejo_instance.add_metric(df, Platforms.FORGEJO, Metrics.ISSUE)
forgejo_instance.add_metric(df, Platforms.FORGEJO, Metrics.PULL_REQUEST)

# Save updated CSV
df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

# Fetching Repositories from Gitlab

In [ ]:
# Fetch a given number of pages of 100 Gitlab repositories
data = gitlab_instance.fetch_repositories(5000)
file_name = "Gitlab_data/gitlab_5k_repositories.csv"

# Save the gathered data to a CSV file
if data:
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False)
    logger.info(f"Saved repositories to '{file_name}'.")
else:
    logger.error("No repositories were fetched.")

### Select Clonable GitLab repositories and add Git metrics to the dataframe

In [ ]:
input_file = "Gitlab_data/gitlab_5k_repositories.csv"
output_file = "Gitlab_data/gitlab_repositories_metrics.csv"

df = pd.read_csv(input_file)
final_df = pd.read_csv(output_file)

df = gitlab_instance.select_clonable_repositories(df, Platforms.GITLAB, 250, final_df)

df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

### Adding Platform specific metrics to the dataframe

In [ ]:
input_file = "Gitlab_data/gitlab_repositories_metrics.csv"
output_file = "Gitlab_data/gitlab_repositories_metrics.csv"

# Load the csv file that contains the repositories
df = pd.read_csv(input_file)

# Add chosen metric for each repository of the DataFrame
gitlab_instance.add_metric(df, Metrics.ISSUE)
gitlab_instance.add_metric(df, Metrics.PULL_REQUEST)

# Save updated CSV
df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

# Fetching Repositories from Bitbucket

In [ ]:
# Fetch a given number of pages of 100 Bitbucket repositories
data = bitbucket_instance.fetch_repositories(100)
file_name = "Bitbucket_data/bitbucket_10k_repositories.csv"

# Save the gathered data to a CSV file
if data:
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False)
    logger.info(f"Saved repositories to '{file_name}'.")
else:
    logger.error("No repositories were fetched.")

### Sample 1000 clonable repositories

In [ ]:
input_file = "Bitbucket_data/bitbucket_10k_repositories.csv"
output_file = "Bitbucket_data/bitbucket_1k_repositories.csv"

df = pd.read_csv(input_file)
df = bitbucket_instance.select_clonable_repositories(df, Platforms.BITBUCKET, 1000)

df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

### Cloning Bitbucket repositories and adding Git metrics to the dataframe

In [ ]:
input_file = "Bitbucket_data/bitbucket_10k_repositories.csv"
output_file = "Bitbucket_data/bitbucket_repositories_metrics.csv"
# Load the csv file that contains the repositories
df = pd.read_csv(input_file)

df = bitbucket_instance.select_clonable_repositories(df, Platforms.BITBUCKET, 500)

# Save updated CSV
df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

### Adding Platform specific metrics to the dataframe

In [ ]:
input_file = "Bitbucket_data/bitbucket_repositories_metrics.csv"
output_file = "Bitbucket_data/bitbucket_repositories_metrics.csv"

# Load the csv file that contains the repositories
df = pd.read_csv(input_file)

# Add chosen metric for each repository of the DataFrame
bitbucket_instance.add_metric(df, Platforms.BITBUCKET, Metrics.STAR)
bitbucket_instance.add_metric(df, Platforms.BITBUCKET, Metrics.FORK)
bitbucket_instance.add_metric(df, Platforms.BITBUCKET, Metrics.PULL_REQUEST)
#bitbucket_instance.add_metric(df, Platforms.BITBUCKET, Metrics.ISSUE)
#gitlab_instance.add_metric(df, Metrics.COMMIT)
#gitlab_instance.add_metric(df, Metrics.CONTRIBUTOR)
#gitlab_instance.add_metric(df, Metrics.LANGUAGE)
#gitlab_instance.add_metric(df, Metrics.LICENSE)

# Save updated CSV
df.to_csv(output_file, index=False)
logger.info(f"Done! Updated file saved as {output_file}")

# First Statistical analysis of the data

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_30days_repositories.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_30days_repositories.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_30days_repositories.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_30days_repositories.csv")
df_github = pd.read_csv("Github_data/github_repositories.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories.csv")

get_most_present_owner(df_github, df_gitlab, df_gitea, df_forgejo)
get_unique_owner_number(df_github, df_gitlab, df_gitea, df_forgejo)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_15days_repositories.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_15days_repositories.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_15days_repositories.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_15days_repositories.csv")

get_most_present_owner(df_github, df_gitlab, df_gitea, df_forgejo)
get_unique_owner_number(df_github, df_gitlab, df_gitea, df_forgejo)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_7days_repositories.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_7days_repositories.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_7days_repositories.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_7days_repositories.csv")

get_most_present_owner(df_github, df_gitlab, df_gitea, df_forgejo)
get_unique_owner_number(df_github, df_gitlab, df_gitea, df_forgejo)

### Plotting the creation date of the repositories

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics_with_date_bias.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics_with_date_bias.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics_with_date_bias.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics_with_date_bias.csv")

# Create the plot
platform_analysis.plot_step_lines(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.CREATED)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_30days_repositories.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_30days_repositories.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_30days_repositories.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_30days_repositories.csv")

# Create the plot
platform_analysis.plot_step_lines(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.CREATED)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_15days_repositories.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_15days_repositories.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_15days_repositories.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_15days_repositories.csv")

# Create the plot
platform_analysis.plot_step_lines(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.CREATED)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_7days_repositories.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_7days_repositories.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_7days_repositories.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_7days_repositories.csv")

# Create the plot
platform_analysis.plot_step_lines(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.CREATED)

### Plotting the last update date of the repositories

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_30days_repositories.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_30days_repositories.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_30days_repositories.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_30days_repositories.csv")

# Create the plot
platform_analysis.plot_step_lines(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.UPDATED)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_15days_repositories.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_15days_repositories.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_15days_repositories.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_15days_repositories.csv")

# Create the plot
platform_analysis.plot_step_lines(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.UPDATED)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_7days_repositories.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_7days_repositories.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_7days_repositories.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_7days_repositories.csv")

# Create the plot
platform_analysis.plot_step_lines(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.UPDATED)

### Checking the distribution of the alphanumeric metrics

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_alphanumeric_distribution(Metrics.MAIN_LANGUAGE,df_github, df_gitlab, df_gitea, df_forgejo)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_alphanumeric_distribution(Metrics.LANGUAGE_DISTRIBUTION,df_github, df_gitlab, df_gitea, df_forgejo)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_alphanumeric_distribution(Metrics.LICENSE,df_github, df_gitlab, df_gitea, df_forgejo)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.COMMIT)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.SIZE)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.CONTRIBUTOR)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.BRANCH)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.ISSUE)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.PULL_REQUEST)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.CREATED)

# Correlation between our metrics

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

print("GITHUB")
#platform_analysis.create_correlation_matrix(df_github)
print("GITLAB")
#platform_analysis.create_correlation_matrix(df_gitlab)
print("GITEA")
#platform_analysis.create_correlation_matrix(df_gitea)
print("FORGEJO")
platform_analysis.create_correlation_matrix(df_forgejo)

### Lorentz curves to visualize check how important the biggest repositories are in the dataset

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

print("GITHUB")
platform_analysis.plot_lorenz_curve(df_github['#commits'])
print("GITLAB")
platform_analysis.plot_lorenz_curve(df_gitlab['#commits'])
print("GITEA")
platform_analysis.plot_lorenz_curve(df_gitea['#commits'])
print("FORGEJO")
platform_analysis.plot_lorenz_curve(df_forgejo['#commits'])

# Statistical analysis using propensity score matching

### Age comparison

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.CREATED)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_gitea, df_forgejo, "Gitea", Metrics.CREATED.value, ["#branches", "#commits", "#contributors", "size"], caliper=0.0001) #ajouter les dates de création

### Commit count comparison

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.COMMIT)

In [ ]:
# Load the dataset
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")

# Filter rows where #commits > 10000
filtered_df = df_gitlab[df_gitlab['#commits'] > 10000]

# Calculate statistics on #contributors
max_contributors = filtered_df['#contributors'].max()
mean_contributors = filtered_df['#contributors'].mean()
median_contributors = filtered_df['#contributors'].median()

# Display the results
print(f"length of filtered_df: {len(filtered_df)}")
print(f"Max #contributors: {max_contributors}")
print(f"Mean #contributors: {mean_contributors}")
print(f"Median #contributors: {median_contributors}")


In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitlab, "GitHub", Metrics.COMMIT.value, ["#branches", "#contributors", "size", "created"], caliper=0.00001) #ajouter les dates de création

### Commit count comparison

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.CONTRIBUTOR)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_gitea, df_forgejo, "Gitea", Metrics.CONTRIBUTOR.value, ["#branches", "#commits", "size", "created"], caliper=0.001) #ajouter les dates de création

### Size comparison

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.SIZE)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_gitea, df_forgejo, "Gitea", Metrics.SIZE.value, ["#branches", "#commits", "#contributors", "created"], caliper=0.1) #ajouter les dates de création

### Stars Comparison

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.STAR)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_gitea, df_forgejo, "Gitea", Metrics.STAR.value, ["#pull_requests", "#issues", "#forks"], caliper=0.01)

### Issues Comparison

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.ISSUE)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_gitea, df_forgejo, "Gitea", Metrics.ISSUE.value, ["#pull_requests", "#stars", "#forks"], caliper=0.001)

### Pull requests comparison

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.PULL_REQUEST)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_gitea, df_forgejo, "Gitea", Metrics.PULL_REQUEST.value, ["#issues", "#stars", "#forks"], caliper=0.001)

# AUTRES

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitlab, "GitHub", Metrics.COMMIT.value, ["#branches", "#contributors", "size", "created"], caliper=0.001) #ajouter les dates de création

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_bitbucket = pd.read_csv("Bitbucket_data/bitbucket_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_bitbucket, Metrics.SIZE, scale="log", max_difference=0.1)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitea, Metrics.SIZE, scale="log", max_difference=0.1)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_forgejo, Metrics.SIZE, scale="log", max_difference=0.1)

### Commit comparison

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitlab, Metrics.COMMIT, scale="log", max_difference=0.1)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_bitbucket = pd.read_csv("Bitbucket_data/bitbucket_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_bitbucket, Metrics.COMMIT, scale="log", max_difference=0.1)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitea, Metrics.COMMIT, scale="log", max_difference=0.1)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_forgejo, Metrics.COMMIT, scale="log", max_difference=0.1)

### Issue comparison

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitlab, Metrics.ISSUE, scale="log", max_difference=0.1)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitea, Metrics.ISSUE, scale="log", max_difference=0.1)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_forgejo, Metrics.ISSUE, scale="log", max_difference=0.1)

### Pull request comparison

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitlab, Metrics.PULL_REQUEST, scale="log", max_difference=0.1)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitea, Metrics.PULL_REQUEST, scale="linear", max_difference=0.1)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_forgejo, Metrics.PULL_REQUEST, scale="log", max_difference=0.1)

# Exemple de présentation des résultats

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.COMMIT)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

print("GITHUB")
platform_analysis.create_correlation_matrix(df_github)
print("GITLAB")
platform_analysis.create_correlation_matrix(df_gitlab)
print("GITEA")
platform_analysis.create_correlation_matrix(df_gitea)
print("FORGEJO")
platform_analysis.create_correlation_matrix(df_forgejo)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

print("GITHUB")
platform_analysis.plot_lorenz_curve(df_github['#commits'])
print("GITLAB")
platform_analysis.plot_lorenz_curve(df_gitlab['#commits'])
print("GITEA")
platform_analysis.plot_lorenz_curve(df_gitea['#commits'])
print("FORGEJO")
platform_analysis.plot_lorenz_curve(df_forgejo['#commits'])

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitlab, "GitHub", Metrics.COMMIT.value, ["#branches", "size", "#contributors", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitea, "GitHub", Metrics.COMMIT.value, ["#branches", "size", "#contributors", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_forgejo, "GitHub", Metrics.COMMIT.value, ["#branches", "size", "#contributors", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_gitlab, df_gitea, "GitLab", Metrics.COMMIT.value, ["#branches", "size", "#contributors", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_gitlab, df_forgejo, "GitLab", Metrics.COMMIT.value, ["#branches", "size", "#contributors", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_gitea, df_forgejo, "Gitea", Metrics.COMMIT.value, ["#branches", "size", "#contributors", "created"]) #ajouter les dates de création

## Contributors

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_forgejo = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Create the plot
platform_analysis.plot_numeric_distribution(df_github, df_gitlab, df_gitea, df_forgejo, Metrics.CONTRIBUTOR)

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitlab, "GitHub", Metrics.CONTRIBUTOR.value, ["#branches", "size", "#commits", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitea, "GitHub", Metrics.CONTRIBUTOR.value, ["#branches", "size", "#commits", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_1 = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_2 = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_1, df_2, "GitHub", Metrics.CONTRIBUTOR.value, ["#branches", "size", "#commits", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_1 = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_2 = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_1, df_2, "GitLab", Metrics.CONTRIBUTOR.value, ["#branches", "size", "#commits", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_1 = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")
df_2 = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_1, df_2, "GitLab", Metrics.CONTRIBUTOR.value, ["#branches", "size", "#commits", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_1 = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")
df_2 = pd.read_csv("Forgejo_data/forgejo_repositories_metrics.csv")

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_1, df_2, "Gitea", Metrics.CONTRIBUTOR.value, ["#branches", "size", "#commits", "created"]) #ajouter les dates de création

## Test en réduisant la quantité d'input pour le matching

In [ ]:
from sklearn.utils import shuffle

# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")

# Shuffle and take a subsample of 750 rows
df_github = shuffle(df_github, random_state=42).sample(n=750)
df_gitlab = shuffle(df_gitlab, random_state=42).sample(n=750)

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitlab, "GitHub", Metrics.CONTRIBUTOR.value, ["#branches", "size", "#commits", "created"]) #ajouter les dates de création

In [ ]:
from sklearn.utils import shuffle

# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitlab = pd.read_csv("Gitlab_data/gitlab_repositories_metrics.csv")

# Shuffle and take a subsample of 750 rows
df_github = shuffle(df_github, random_state=42).sample(n=500)
df_gitlab = shuffle(df_gitlab, random_state=42).sample(n=500)

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitlab, "GitHub", Metrics.CONTRIBUTOR.value, ["#branches", "size", "#commits", "created"]) #ajouter les dates de création

In [ ]:
r# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")

# Shuffle and take a subsample of 750 rows
df_github = shuffle(df_github, random_state=42).sample(n=750)
df_gitea = shuffle(df_gitea, random_state=42).sample(n=750)

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitea, "GitHub", Metrics.CONTRIBUTOR.value, ["#branches", "size", "#commits", "created"]) #ajouter les dates de création

In [ ]:
# Load Data
df_github = pd.read_csv("Github_data/github_repositories_metrics.csv")
df_gitea = pd.read_csv("Gitea_data/gitea_repositories_metrics.csv")

# Shuffle and take a subsample of 750 rows
df_github = shuffle(df_github, random_state=42).sample(n=500)
df_gitea = shuffle(df_gitea, random_state=42).sample(n=500)

# Perform matching and create the plot
platform_analysis.propensity_score_matching(df_github, df_gitea, "GitHub", Metrics.CONTRIBUTOR.value, ["#branches", "size", "#commits", "created"]) #ajouter les dates de création

In [ ]:
import requests
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

# Step 1: Call the API
url = "https://repos.ecosyste.ms/api/v1/hosts/GitHub/repositories?per_page=100"
response = requests.get(url)

# Check for successful response
if response.status_code != 200:
    raise Exception(f"API request failed with status code {response.status_code}")

data = response.json()

# Step 2: Extract "last_synced_at" values
last_synced_dates = []
for repo in data:
    last_synced_at = repo.get("last_synced_at")
    if last_synced_at:
        # Convert to datetime object
        dt = datetime.fromisoformat(last_synced_at.replace("Z", "+00:00"))
        last_synced_dates.append(dt)

# Step 3: Create a plot showing the distribution
# We'll use pandas to group by date
df = pd.DataFrame({"last_synced_at": last_synced_dates})
df["date_only"] = df["last_synced_at"].dt.date

# Count frequencies by day
date_counts = df["date_only"].value_counts().sort_index()

# Calculate percentage before 2025
cutoff_date = datetime(2025, 1, 1, tzinfo=last_synced_dates[0].tzinfo)  # match timezone
total = len(last_synced_dates)
before_2025 = sum(dt < cutoff_date for dt in last_synced_dates)
percentage = (before_2025 / total) * 100 if total > 0 else 0

print(f"{percentage:.2f}% of repositories have a 'last_synced_at' date before 2025.")

# Plotting
plt.figure(figsize=(12, 6))
date_counts.plot(kind='bar')
plt.xlabel("Date")
plt.ylabel("Nombre de dépôts")
plt.xticks(rotation=45)
plt.tight_layout()
#plt.savefig("Figures/last_synced_distribution.png", dpi=300)
plt.show()
